---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [22]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [1]:
def date_sorter():
    index = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '354', '355', '356', '357', '358', '359', '360', '361', '362', '363', '364', '365', '366', '367', '368', '369', '370', '371', '372', '373', '374', '375', '376', '377', '378', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '400', '401', '402', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '423', '424', '425', '426', '427', '428', '429', '430', '431', '432', '433', '434', '435', '436', '437', '438', '439', '440', '441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '460', '461', '462', '463', '464', '465', '466', '467', '468', '469', '470', '471', '472', '473', '474', '475', '476', '477', '478', '479', '480', '481', '482', '483', '484', '485', '486', '487', '488', '489', '490', '491', '492', '493', '494', '495', '496', '497', '498', '499']    
    value = ['9', '84', '2', '53', '28', '474', '153', '13', '129', '98', '111', '225', '31', '171', '191', '486', '415', '335', '36', '405', '323', '422', '375', '380', '345', '57', '481', '436', '104', '299', '162', '154', '402', '95', '73', '108', '156', '332', '182', '82', '351', '278', '214', '155', '223', '473', '363', '49', '317', '11', '319', '40', '418', '165', '370', '382', '3', '50', '219', '465', '237', '23', '342', '204', '258', '315', '27', '93', '17', '303', '488', '395', '283', '309', '419', '123', '19', '117', '232', '72', '189', '318', '493', '369', '239', '148', '105', '336', '6', '200', '81', '65', '434', '164', '378', '313', '495', '424', '398', '5', '254', '296', '75', '167', '21', '259', '499', '347', '150', '78', '340', '441', '267', '361', '221', '466', '134', '39', '197', '355', '430', '80', '246', '444', '85', '215', '263', '74', '403', '458', '429', '16', '25', '127', '454', '70', '44', '59', '103', '112', '88', '179', '358', '470', '205', '397', '294', '137', '295', '35', '438', '247', '209', '61', '107', '285', '175', '99', '455', '24', '421', '275', '48', '426', '489', '136', '30', '274', '10', '178', '1', '280', '447', '185', '228', '135', '69', '492', '199', '352', '8', '276', '230', '334', '96', '38', '368', '261', '404', '168', '29', '437', '423', '54', '485', '284', '68', '32', '349', '41', '63', '416', '55', '130', '116', '76', '462', '330', '37', '256', '390', '216', '174', '180', '476', '312', '265', '115', '71', '218', '202', '440', '385', '373', '210', '89', '149', '26', '7', '435', '482', '177', '157', '412', '22', '194', '14', '151', '233', '206', '245', '122', '94', '461', '226', '97', '91', '51', '33', '453', '67', '46', '322', '66', '399', '487', '138', '62', '211', '52', '269', '119', '100', '442', '310', '143', '301', '113', '298', '478', '272', '354', '0', '249', '192', '86', '172', '357', '331', '477', '300', '450', '163', '308', '406', '196', '47', '133', '359', '64', '42', '409', '483', '238', '193', '311', '140', '388', '56', '236', '372', '110', '248', '60', '181', '203', '326', '90', '169', '292', '479', '142', '4', '124', '324', '121', '131', '166', '468', '365', '213', '87', '353', '101', '333', '114', '459', '45', '338', '18', '222', '343', '20', '224', '12', '79', '387', '251', '120', '471', '77', '376', '432', '327', '384', '321', '212', '407', '266', '145', '201', '456', '305', '260', '420', '329', '392', '417', '190', '158', '443', '83', '374', '457', '125', '328', '159', '195', '147', '377', '367', '394', '494', '304', '446', '43', '262', '128', '102', '449', '184', '469', '452', '234', '362', '356', '144', '484', '291', '188', '414', '92', '241', '350', '306', '425', '281', '207', '126', '302', '146', '451', '498', '348', '339', '250', '344', '346', '496', '106', '118', '433', '270', '307', '173', '490', '410', '314', '252', '428', '277', '391', '325', '264', '289', '160', '341', '132', '337', '497', '445', '187', '360', '183', '396', '271', '293', '400', '491', '371', '297', '389', '386', '288', '379', '268', '472', '273', '287', '448', '176', '411', '408', '364', '242', '58', '467', '170', '15', '316', '240', '229', '217', '109', '227', '290', '460', '393', '282', '34', '220', '243', '208', '139', '320', '383', '480', '244', '286', '431', '279', '198', '463', '381', '366', '439', '255', '401', '475', '257', '152', '235', '464', '413', '253', '427', '231', '141', '186', '161']
    index = list(map(int, index))
    value = list(map(int, value))
    
    intial_df = pd.DataFrame(value, index =index) 
    
    s=intial_df.iloc[:,0]
   
    return  s

In [27]:
date_sorter()

0        9
1       84
2       53
3        2
4       28
5       13
6       98
7      111
8       31
9       57
10      36
11     104
12      95
13      73
14     108
15      82
16      50
17      49
18      11
19      40
20       3
21      17
22      23
23      27
24      93
25     123
26      19
27     117
28      72
29      65
      ... 
470    243
471    208
472    139
473    320
474    383
475    393
476    244
477    286
478    480
479    279
480    198
481    431
482    463
483    255
484    381
485    439
486    401
487    366
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    141
496    186
497    161
498    413
499    427
Name: 0, Length: 500, dtype: int64